In [1]:
from PIL import Image
from torchvision import transforms

import numpy as np

img = Image.open("/Users/prajjvalgovil/researchmethodology/collaborative_cnn_team04/data/test/2.jpg").convert("RGB")
t = transforms.ToTensor()

tensor = t(img)

print("Tensor OK", tensor.shape)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tensor OK torch.Size([3, 299, 296])


In [2]:
import numpy
import torch
import torchvision

print("Numpy:", numpy.__version__)
print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)

Numpy: 1.23.5
Torch: 2.2.2
Torchvision: 0.17.2


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import json
import numpy

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
import sys
sys.path.append("../models")

from model_v1 import CnnModel

model = CnnModel()
model.load_state_dict(torch.load("../models/model_v1.pth", map_location=device))
model.to(device)
model.eval()

print("Model Loaded Successfully!")

Model Loaded Successfully!


In [6]:
class User2TestDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_files = [
            f for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder_path, img_name)
        
        img = Image.open(img_path).convert("RGB")
        
        if self.transform:
            img = self.transform(img)
        
        return img, img_name

In [7]:
# !ls ../data/test


In [8]:
test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])


test_folder = "../data/test"

test_data_user2 = User2TestDataset(test_folder, transform=test_transform)
test_loader_user2 = DataLoader(test_data_user2, batch_size=16, shuffle=False)

len(test_data_user2)

12500

In [9]:
predictions = []
total_images = 0

with torch.no_grad():
    for images, names in test_loader_user2:
        images = images.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        for name, pred in zip(names, predicted):
            predictions.append({
                "image": name,
                "predicted_class": int(pred.item())
            })

        total_images += len(names)

print("Inference Completed!")

Inference Completed!


In [10]:
results = {
    "total_images": total_images,
    "predictions": predictions
}

os.makedirs("../results", exist_ok=True)

with open("../results/test_v1_user2.json", "w") as f:
    json.dump(results, f, indent=4)

print("Saved to ../results/test_v1_user2.json")

Saved to ../results/test_v1_user2.json


In [11]:
predictions[:5]

[{'image': '9733.jpg', 'predicted_class': 0},
 {'image': '63.jpg', 'predicted_class': 0},
 {'image': '6400.jpg', 'predicted_class': 1},
 {'image': '823.jpg', 'predicted_class': 1},
 {'image': '4217.jpg', 'predicted_class': 1}]

In [12]:
with open("../results/test_v1_user2.json", "w") as f:
    json.dump(results, f, indent=4)